In [1]:
# Imports

import numpy as np
import logging; logger = logging.getLogger(__name__)


from opendrift.models.oceandrift import OceanDrift, Lagrangian3DArray
from opendrift.models.leeway import Leeway # For Search and Rescure. May not be relevant
from opendrift.models.plastdrift import PlastDrift
from opendrift.models.sedimentdrift import SedimentDrift # Looks most promising
from opendrift.models.radionuclides import Radionuclide

from opendrift.readers import reader_netCDF_CF_generic

from pprint import pprint
from opendrift.readers import reader_global_landmask


from datetime import datetime

from opendrift.models.basemodel import OpenDriftSimulation
from opendrift.elements import LagrangianArray


### Define the model

Here, we define the class *GhostNetElement*, which is supposed to represent a single ghost net in the ocean. Then, the class *GhostNetDrift* handles the simulation. <br>

As of now, I simply copied the code from `opendrift.models.sedimentdrift.SedimentDrift`. We should also check out `opendrift.models.radionnuclides.Radionnuclide` as it also incorporates diameter, density, etc. Then, the RadionNuclideDrift shows how these properties can be handled in the simulation. <br>

The most important parts here are the `variables` field in GhostNetElement, the `required_variables` field in GhostNetDrift and the `update()` function in GhostNetDrift.

In [2]:
class GhostNetElement(Lagrangian3DArray):
    variables = Lagrangian3DArray.add_variables([
        ('settled', {'dtype': np.uint8,  # 0 is active, 1 is settled
                     'units': '1',
                     'default': 0}),
        ('terminal_velocity', {'dtype': np.float32,
                               'units': 'm/s',
                               'default': -0.001})  # 1 mm/s negative buoyancy
        ])

# Differences between RadionNuclide and Sediment:
# diameter in radionuclide: BUT only needed to update terminal velocity (don't include)
# density: -------||---------


class GhostNetDrift(OceanDrift):
    """Model for sediment drift, under development
    """

    ElementType = GhostNetElement

    required_variables = {
        'x_sea_water_velocity': {'fallback': 0},
        'y_sea_water_velocity': {'fallback': 0},
        'upward_sea_water_velocity': {'fallback': 0},
        'x_wind': {'fallback': 0},
        'y_wind': {'fallback': 0},
        'sea_surface_wave_stokes_drift_x_velocity': {'fallback': 0},
        'sea_surface_wave_stokes_drift_y_velocity': {'fallback': 0},
        'sea_surface_wave_period_at_variance_spectral_density_maximum': {'fallback': 0},
        'sea_surface_wave_mean_period_from_variance_spectral_density_second_frequency_moment': {'fallback': 0},
        'land_binary_mask': {'fallback': None},
        'ocean_vertical_diffusivity': {'fallback': 0.02},
        'ocean_mixed_layer_thickness': {'fallback': 50},
        'sea_floor_depth_below_sea_level': {'fallback': 0},
        }

    def __init__(self, *args, **kwargs):
        """ Constructor of SedimentDrift module
        """

        super(GhostNetDrift, self).__init__(*args, **kwargs)

        # By default, sediments do not strand towards coastline
        # TODO: A more sophisticated stranding algorithm is needed
        self._set_config_default('general:coastline_action', 'previous')

        # Vertical mixing is enabled as default
        self._set_config_default('drift:vertical_mixing', True)

    def update(self):
        """Update positions and properties of sediment particles.
        """

        # Advecting here all elements, but want to soon add
        # possibility of not moving settled elements, until
        # they are resuspended. May then need to send a boolean
        # array to advection methods below
        self.advect_ocean_current()

        self.vertical_advection()

        self.advect_wind()  # Wind shear in upper 10cm of ocean

        self.stokes_drift()

        #self.vertical_mixing()  # Including buoyancy and settling

        self.resuspension()

    def bottom_interaction(self, seafloor_depth):
        """Sub method of vertical_mixing, determines settling"""
        # Elements at or below seafloor are settled, by setting
        # self.elements.moving to 0.
        # These elements will not move until eventual later resuspension.
        settling = np.logical_and(self.elements.z <= seafloor_depth, self.elements.moving==1)
        if np.sum(settling) > 0:
            logger.debug('Settling %s elements at seafloor' % np.sum(settling))
            self.elements.moving[settling] = 0

    def resuspension(self):
        """Resuspending elements if current speed > .5 m/s"""
        resuspending = np.logical_and(self.current_speed()>.5, self.elements.moving==0)
        if np.sum(resuspending) > 0:
            # Allow moving again
            self.elements.moving[resuspending] = 1
            # Suspend 1 cm above seafloor
            self.elements.z[resuspending] = self.elements.z[resuspending] + .01


In [3]:
o = GhostNetDrift(loglevel=0)

14:53:02 DEBUG   opendrift.models.basemodel:618: Adding 17 config items from basemodel
14:53:02 DEBUG   opendrift.models.basemodel:618: Adding 6 config items from basemodel
14:53:02 DEBUG   opendrift.models.basemodel:618: Adding 26 config items from basemodel
14:53:02 INFO    opendrift.models.basemodel:532: OpenDriftSimulation initialised (version 1.10.6 / v1.10.6-5-ged278b2)
14:53:02 DEBUG   opendrift.models.basemodel:618: Adding 14 config items from oceandrift
14:53:02 DEBUG   opendrift.models.basemodel:630:   Overwriting config item seed:z


In [4]:
pprint(o.required_variables)

{'land_binary_mask': {'fallback': None},
 'ocean_mixed_layer_thickness': {'fallback': 50},
 'ocean_vertical_diffusivity': {'fallback': 0.02},
 'sea_floor_depth_below_sea_level': {'fallback': 0},
 'sea_surface_wave_mean_period_from_variance_spectral_density_second_frequency_moment': {'fallback': 0},
 'sea_surface_wave_period_at_variance_spectral_density_maximum': {'fallback': 0},
 'sea_surface_wave_stokes_drift_x_velocity': {'fallback': 0},
 'sea_surface_wave_stokes_drift_y_velocity': {'fallback': 0},
 'upward_sea_water_velocity': {'fallback': 0},
 'x_sea_water_velocity': {'fallback': 0},
 'x_wind': {'fallback': 0},
 'y_sea_water_velocity': {'fallback': 0},
 'y_wind': {'fallback': 0}}


### Add Readers. There are two readers of interest: 
- Generic reader that retrieves ocean data for simulating currents
- Landmask reader for land data (where ocean meets land). This file is simply 3D tensor (matrix) containing 1 where there is land and 0 where there is ocean <br>

Each reader needs a filename, that is either a local file or URL. Here I use the URL provided [here](https://opendrift.github.io/tutorial.html#import-a-specific-model-for-the-relevant-application) <br>

**They are accessed in the following manner:**

In [5]:
# Generic reader
reader_norkyst = reader_netCDF_CF_generic.Reader(
    filename='https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be'
)

# Landmask reader. 
reader_landmask = reader_global_landmask.Reader()

14:53:04 INFO    opendrift.readers.reader_netCDF_CF_generic:138: Opening dataset: https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be
14:53:27 DEBUG   opendrift.readers.reader_netCDF_CF_generic:155: Finding coordinate variables.
14:53:27 DEBUG   opendrift.readers.reader_netCDF_CF_generic:355: Skipped variables without standard_name: ['angle', 'tke', 'ubar', 'vbar']
14:53:27 DEBUG   opendrift.readers.basereader.variables:546: Setting buffer size 25 for reader https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be, assuming a maximum average speed of 5 m/s and time span of 1:00:00
14:53:27 DEBUG   opendrift.readers.basereader:168: Variable mapping: ['sea_floor_depth_below_sea_level'] -> ['land_binary_mask'] is not activated
14:53:27 DEBUG   opendrift.readers.basereader:168: Variable mapping: ['sea_floor_depth_below_sea_level'] -> ['land_binary_mask'] is not activated


In [6]:
o.add_reader([reader_landmask, reader_norkyst])

14:53:33 DEBUG   opendrift.models.basemodel:945: Added reader global_landmask
14:53:33 DEBUG   opendrift.readers.basereader.variables:546: Setting buffer size 7 for reader https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be, assuming a maximum average speed of 1 m/s and time span of 1:00:00
14:53:33 DEBUG   opendrift.models.basemodel:945: Added reader https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be


### Seeding elements
To seed an element simply means releasing it into the ocean (for simulation). <br>

**Important:**
- If `lon` and `lat` are given as arrays of two elements, the GhostNet is dropped randomly at the line between them. Here, the radius can incorporate the uncertainty of the line's width. 
- If `time` is given as an array, then the GhostNet is dropped randomly in the timeinterval. 

In [7]:
# These are the standard arguments. 
# number GhostNet is dropped at longitude 4.3, lattitude 60 within a radius of 1000 meters. 
o.seed_elements(lon=4.5, lat=60, radius=1000, number=10, time=reader_norkyst.start_time)

In [8]:
# Inspect properties of the GhostNet dropped
o.elements_scheduleds

ID: [ 1  2  3  4  5  6  7  8  9 10]
status: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
moving: 1
age_seconds: 0.0
origin_marker: 0
lon: [4.5316153 4.507174  4.517544  4.5401607 4.533473  4.4824843 4.5170336
 4.4972878 4.49815   4.5073566]
lat: [60.00129  60.013054 60.00683  60.001087 60.00398  60.002995 60.01341
 59.998158 60.00281  59.992332]
z: 0.0
wind_drift_factor: 0.02
current_drift_factor: 1.0
terminal_velocity: -0.001
settled: 0

### Configurating the simulation

We can also provide custom configurations to the simulation, that may suit GhostNets better.

In [9]:
# Parameters available to configurate:
o.list_configspec()

general:use_auto_landmask           [True] bool   A built-in GSHHG glo...
general:coastline_action            [previous] enum  ['none', 'stranding', 'previous'] None means that obje...
general:time_step_minutes           [60] float min: 0.01, max: 1440 [minutes] Calculation time ste...
general:time_step_output_minutes    [None] float min: 1, max: 1440 [minutes] Output time step, i....
seed:ocean_only                     [True] bool   If True, elements se...
seed:number                         [1] int   min: 1, max: 100000000 [1] The number of elemen...
drift:max_age_seconds               [None] float min: 0, max: inf [seconds] Elements will be dea...
drift:advection_scheme              [euler] enum  ['euler', 'runge-kutta', 'runge-kutta4'] Numerical advection ...
drift:current_uncertainty           [0] float min: 0, max: 5 [m/s] Add gaussian perturb...
drift:current_uncertainty_uniform   [0] float min: 0, max: 5 [m/s] Add gaussian perturb...
drift:horizontal_diffusivity        [0] floa

### Running the model

In [10]:
# Variables the simulation needs
o.required_variables

{'x_sea_water_velocity': {'fallback': 0},
 'y_sea_water_velocity': {'fallback': 0},
 'upward_sea_water_velocity': {'fallback': 0},
 'x_wind': {'fallback': 0},
 'y_wind': {'fallback': 0},
 'sea_surface_wave_stokes_drift_x_velocity': {'fallback': 0},
 'sea_surface_wave_stokes_drift_y_velocity': {'fallback': 0},
 'sea_surface_wave_period_at_variance_spectral_density_maximum': {'fallback': 0},
 'sea_surface_wave_mean_period_from_variance_spectral_density_second_frequency_moment': {'fallback': 0},
 'land_binary_mask': {'fallback': None},
 'ocean_vertical_diffusivity': {'fallback': 0.02},
 'ocean_mixed_layer_thickness': {'fallback': 50},
 'sea_floor_depth_below_sea_level': {'fallback': 0}}

In [11]:
# Hover over run to see documentation
o.run(time_step=3600, steps=1)

14:53:37 DEBUG   opendrift.models.basemodel:2533: 
------------------------------------------------------
Software and hardware:
  OpenDrift version 1.10.6
  Platform: Darwin, 22.3.0
  64.0 GB memory
  10 processors (i386)
  NumPy version 1.24.2
  SciPy version 1.10.1
  Matplotlib version 3.7.1
  NetCDF4 version 1.6.3
  Xarray version 2023.3.0
  Python version 3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]
------------------------------------------------------

14:53:37 DEBUG   opendrift.models.basemodel:2550: No output file is specified, neglecting export_buffer_length
14:53:37 INFO    opendrift.models.basemodel:2579: Fallback values will be used for the following variables which have no readers: 
14:53:37 INFO    opendrift.models.basemodel:2582: 	sea_surface_wave_stokes_drift_x_velocity: 0.000000
14:53:37 INFO    opendrift.models.basemodel:2582: 	sea_surface_wave_stokes_drift_y_velocity: 0.000000
14:53:37 INFO    opendrift.models.basemodel:2582: 	sea

14:53:38 DEBUG   opendrift.models.basemodel:3168: Setting up map: corners=None, fast=False, lscale=None
14:53:38 DEBUG   opendrift.readers.reader_global_landmask:83: Adding GSHHG shapes from cartopy, scale: f, extent: (4.082484340667724, 4.941754722595214, 59.77540664672852, 60.21340866088866)..
14:54:08 DEBUG   opendrift.readers.reader_global_landmask:83: Adding GSHHG shapes from cartopy, scale: f, extent: (4.082484340667724, 4.941754722595214, 59.77540664672852, 60.21340866088866)..
14:54:09 DEBUG   opendrift.readers.reader_global_landmask:83: Adding GSHHG shapes from cartopy, scale: f, extent: (4.082484340667724, 4.941754722595214, 59.77540664672852, 60.21340866088866)..


(<GeoAxes: title={'center': 'OpenDrift - GhostNetDrift\n2017-02-20 00:00 to 2017-02-20 01:00 UTC (2 steps)'}>,
 <Figure size 1079.17x1100 with 1 Axes>)

In [12]:
o.plot()

### Testing SINTEF data (not relevant ATM)

In [ ]:
# Change name of time variable
import os
for file in [f'./data/aver-monthly{bool(i)*f"-{i}"}' for i in range(6)]:
    os.system(f'ncrename -v time,time_var {file}.nc {file}-xr.nc')

: 

In [ ]:
norkyst = reader_netCDF_CF_generic.Reader(
    [f'./data/aver-monthly{bool(i)*f"-{i}"}-xr.nc' for i in range(6)]
)

: 

In [ ]:
print(reader_netCDF_CF_generic.Reader('./data/aver-monthly-xr.nc'))

: 

In [ ]:
print(reader_netCDF_CF_generic.Reader('./data/aver-monthly-1-xr.nc'))

: 

In [ ]:
norkyst = reader_netCDF_CF_generic.Reader(
    './data/aver-monthly-xr.nc', standard_name_mapping={'time_var': 'time'}
)

: 

In [ ]:
reader_norkyst.variable_mapping

: 

In [ ]:
from netCDF4 import Dataset

ds = Dataset('./data/aver-monthly.nc')

type(ds.dimensions.values())

dims = ds.dimensions.values()
time = dims.mapping['time']

: 

In [ ]:
time

: 

In [ ]:
ds['time']

: 

In [ ]:
vars = ds.variables.values()
vars

: 

In [ ]:
reader_norkyst = reader_netCDF_CF_generic.Reader(
    'https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be'
)
print(reader_norkyst)

: 

In [ ]:
reader_norkyst.variables

: 

In [ ]:
from datetime import datetime, timedelta
from opendrift.models.shipdrift import ShipDrift

o = ShipDrift()
o.add_readers_from_list(
    ['https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be']
)
#o.disable_vertical_motion()
o.seed_elements(lon=4.85, lat=60, time=datetime.now(), number=10000, radius=1000)

o.run(duration=timedelta(hours=72))
o.animation(filename='animation.mp4')

: 

: 